<a href="https://colab.research.google.com/github/Vel-Murugan-K/customer-support-ticket-triage/blob/main/Week1_Customer_Support_Triage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


df = pd.read_csv("/content/customer_support_tickets.csv")


text_col = [c for c in df.columns if "message" in c.lower()
            or "issue" in c.lower()
            or "description" in c.lower()][0]

time_col = [c for c in df.columns if "date" in c.lower()
            or "time" in c.lower()][0]


df["clean_message"] = (
    df[text_col]
    .astype(str)
    .str.lower()
    .str.replace(r"[^a-z0-9 ]", " ", regex=True)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)


df["issue_type"] = "GENERAL"

df.loc[df["clean_message"].str.contains("payment|paid|card|upi|transaction"), "issue_type"] = "PAYMENT"
df.loc[df["clean_message"].str.contains("login|password|signin|otp"), "issue_type"] = "LOGIN"
df.loc[df["clean_message"].str.contains("delivery|shipping|courier|late"), "issue_type"] = "DELIVERY"
df.loc[df["clean_message"].str.contains("refund|return|money back"), "issue_type"] = "REFUND"
df.loc[df["clean_message"].str.contains("bug|error|crash|problem|not working"), "issue_type"] = "BUG"


priority_map = {
    "PAYMENT": "P0", "BUG": "P0",
    "LOGIN": "P1", "REFUND": "P1",
    "DELIVERY": "P2", "GENERAL": "P3"
}
df["priority"] = df["issue_type"].map(priority_map)

sla_map = {"P0": 4, "P1": 8, "P2": 24, "P3": 72}
df["sla_hours"] = df["priority"].map(sla_map)

df[time_col] = pd.to_datetime(df[time_col], errors="coerce")
df["due_time"] = df[time_col] + pd.to_timedelta(df["sla_hours"], unit="h")

df.to_csv("/content/final_customer_support_output.csv", index=False)

print("SUCCESS")
print("Customer name column preserved (if present)")
print(df["issue_type"].value_counts())

SUCCESS
Customer name column preserved (if present)
issue_type
GENERAL     3701
BUG         3592
DELIVERY     825
REFUND       180
PAYMENT      122
LOGIN         49
Name: count, dtype: int64
